In [ ]:
# Notebook to load markdown content to Azure AI Search
# NOTE: There is no chunking of content, but rather a single PDF page equals a chunk

In [2]:
import os
import concurrent.futures  
from functools import partial  
# Utils
import doc2md_utils

markdown_path = 'markdown'

In [3]:
# Load the Index Scheam and re-create Index
doc2md_utils.create_index()

Dimensions in Embedding Model: 1536
Index gpt4o-extraction deleted successfully.
Index gpt4o-extraction created successfully.


In [4]:
# Retrive all the markdown files and doc_id
doc_id = doc2md_utils.get_doc_id(markdown_path)
markdown_out_dir = os.path.join(markdown_path, doc_id)   

files = os.listdir(markdown_out_dir)  
# Filter out non-txt files (optional)  
txt_files = [f for f in files if f.endswith('.txt')]  

total_files = len(files)
print ('Total Markdown Files:', total_files)


doc_id: a55a8e09-4b40-4b7e-a37b-bc379a9fb48a
Total Markdown Files: 9


In [5]:
# Vectorize the JSON Content in parallel
max_workers = 15

json_out_dir = os.path.join('json', doc_id)
doc2md_utils.ensure_directory_exists(json_out_dir)

partial_process_json = partial(doc2md_utils.process_json, doc_id=doc_id, markdown_out_dir=markdown_out_dir, json_out_dir=json_out_dir)  
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:  
    results = list(executor.map(partial_process_json, files))  
print(results)  

json_files = doc2md_utils.get_all_files(json_out_dir)
total_files = len(json_files)
print ('Total JSON Files:', total_files)


Directory created: json/a55a8e09-4b40-4b7e-a37b-bc379a9fb48a
['1.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt', '9.txt']
Total JSON Files: 9


In [6]:
# Index content to Azure AI Search
doc2md_utils.index_content(json_files)

Documents Indexed successfully.
